In [ ]:
# Importing Library
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
# Reading the training dataset in a dataframe using Pandas
df = pd.read_csv("train.csv")
# Reading the test dataset in a dataframe using Pandas
test = pd.read_csv("test.csv")


In [ ]:

df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)

df['Married'].fillna(df['Married'].mode()[0], inplace=True)

df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)

df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)

cat = ['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Credit_History', 'Property_Area']

for var in cat:
    le = preprocessing.LabelEncoder()
    df[var] = le.fit_transform(df[var].astype('str'))
df.dtypes


In [35]:
from sklearn import metrics
from sklearn.model_selection import KFold



def classification_model(model, data, predictors, outcome):

    model.fit(data[predictors],data[outcome])

    predictions = model.predict(data[predictors])

    accuracy = metrics.accuracy_score(predictions,data[outcome])
    print ("Accuracy : %s" % "{0:.3%}".format(accuracy))


    kf = KFold(data.shape[0], n_folds=5)
    error = []
    for train, test in kf:

        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]

        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
    print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))


    model.fit(data[predictors],data[outcome])

 

In [ ]:
df['Type'] = 'Train'
test['Type'] = 'Test'
fullData = pd.concat([df, test], axis=0, sort=True)


fullData.isnull().sum()


In [37]:
ID_col = ['Loan_ID']
target_col = ["Loan_Status"]
cat_cols = ['Credit_History','Dependents','Gender','Married','Education','Property_Area','Self_Employed']

In [ ]:
fullData['LoanAmount'].fillna(fullData['LoanAmount'].mean(), inplace=True)

fullData['LoanAmount_log'].fillna(
    fullData['LoanAmount_log'].mean(), inplace=True)

fullData['Loan_Amount_Term'].fillna(
    fullData['Loan_Amount_Term'].mean(), inplace=True)

fullData['ApplicantIncome'].fillna(
    fullData['ApplicantIncome'].mean(), inplace=True)
    
fullData['CoapplicantIncome'].fillna(
    fullData['CoapplicantIncome'].mean(), inplace=True)


fullData['Gender'].fillna(fullData['Gender'].mode()[0], inplace=True)
fullData['Married'].fillna(fullData['Married'].mode()[0], inplace=True)
fullData['Dependents'].fillna(fullData['Dependents'].mode()[0], inplace=True)

fullData['Loan_Amount_Term'].fillna(
    fullData['Loan_Amount_Term'].mode()[0], inplace=True)

fullData['Credit_History'].fillna(
    fullData['Credit_History'].mode()[0], inplace=True)


In [ ]:


fullData['TotalIncome'] = fullData['ApplicantIncome'] + \
    fullData['CoapplicantIncome']

fullData['TotalIncome_log'] = np.log(fullData['TotalIncome'])

fullData['TotalIncome_log'].hist(bins=20)


In [ ]:
#create label encoders for categorical features
for var in cat_cols:
    number = LabelEncoder()
    fullData[var] = number.fit_transform(fullData[var].astype('str'))

train_modified = fullData[fullData['Type'] == 'Train']
test_modified = fullData[fullData['Type'] == 'Test']
train_modified["Loan_Status"] = number.fit_transform(
    train_modified["Loan_Status"].astype('str'))


In [ ]:
from sklearn.linear_model import LogisticRegression

predictors_Logistic=['Credit_History','Education','Gender']

x_train = train_modified[list(predictors_Logistic)].values
y_train = train_modified["Loan_Status"].values

x_test=test_modified[list(predictors_Logistic)].values

In [48]:
model = LogisticRegression()

model.fit(x_train)

predicted= model.predict(x_test,y_train)

predicted = number.inverse_transform(predicted)

test_modified['Loan_Status']=predicted

outcome_var = 'Loan_Status'

classification_model(model, df,predictors_Logistic,outcome_var)

test_modified.to_csv("Logistic_Prediction.csv",columns=['Loan_ID','Loan_Status'])

TypeError: LogisticRegression.fit() missing 1 required positional argument: 'y'